**Red Wine Quality Classification**

In this analysis of the work, it will be determined which physiochemical properties make red wine 'good!' by using some machine learning techniques. Content of the paper is starting with introduction of the data set, and some data visualization. Then some classification and one regression technique will be applied to the data and with cross validation accuracy will be evaluated. Lastly, the best estimator will be decided according to cross validation score and conculusion.

-Introduction

-Data Introduction(Data upload and some enhancements)

-Data visualization

-Classification and Cross Validation Results

    1. Logistic Regression
    2. SVM
    3. Decision Tree
    4. Random Forest
    
 -Best Classifier according to classfication accuraies 
 
 -Conculusion

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #plotting
import seaborn as sns #good visualizing
import os
import warnings
warnings.filterwarnings('ignore')
print(os.listdir("../input"))
data = pd.read_csv('../input/winequality-red.csv')
data.columns = data.columns.str.replace(' ','_')
data.info()

**Data Introduction**

In this dataset there are specifically red wine variants of Poteguese "Vinho Verde" wine.  Due to privacy and logistic issues, only physicochemical (inputs) and sensory (the output) variables are available (e.g. there is no data about grape types, wine brand, wine selling price, etc.).
For more information, read [Cortez et al., 2009].

In the data set, there 1599 different wine as row data and 12 features as columns. Furthermore, there is no nun value to deal with it and all values are numeric means that input values are float and only output value is integer.



In [ ]:
data.describe()

The measures of central tendency and variability or distribution are some commonly used measures to define the data set. The measures used to define the central tendency are mean, median and mode. The standard deviations (or variance) are the minimum and maximum values of variables. The table above is a summary of some statistical measures for each numeric predictor of the dataset: 
**count** indicates the number of records for each attribute that corresponds to the number of wines. 
**mean** indicates the average value around which each group of attributes is attested.
**std** indicates the standard deviation of each attribute group, from which we can guess the degree of data dispersion around the average.
**max** and min indicate the attribute that I assume the highest and lowest value for each attribute group.

**Data visualization**

This data set has many different features and it is important to understand relationship between these in order to analyze dataset better. For that reason, correlation map helps to understand these relations in a single representation. Correlation map is made by calculating the covariance of each features with respect to others, then each covariance value is divided by standard deviation of each variables and get results between -1, 0, 1.

-1 means: There is a negative relationship between dependent and independent variables .

0 means: There is no relationship between dependent and independent variables .

1 means: There is a positive relationship between dependent and independent variables .
According to these information, it can be made a good analyze about dataset and columns.

In [ ]:
#correlation map view
data.corr() 
f, ax = plt.subplots(figsize = (10,10))
sns.heatmap(data.corr(), annot = True, linewidths=.5, fmt = ".2f", ax=ax)
plt.show()

According to figure in above;
* Quality has a (+)positive relationship  between alcohol 
* Quality has a (-)negative weak relationship  between volitile_acidicity
* Quality has almost no relationship  between residual_sugar, free_sulfur_dioxide, and pH.(corr =~ 0)

* Alcohol  has a (+)positive relationship  between quality and weakly pH
* Alcohol  has a (-)negative relationship  between density
* Alcohol  has almost no relationship  between fixed_acidicity, residual_sugar, free_sulfur_dioxide, sulphates

* Volitile_acidicity has a weak (+)positive relationship  between pH.
* Volitile_acidicity has a strong (-)negative relationship  between citric_acid
* Volitile_acidicity has weak (-)negative relationship  between fixed_acidicity and sulphates
* Volitile_acidicity has almost no relationship between residual_sugar, chlorides, free_sulfur_dioxide, total_sulfur_dioxide, density

* Density has (+)positive relationship between fixed_acidicity
* Density has (-)negative relationship between density
* Density has almost no relationship between volitile_acidicity, free_sulfur_dioxide, total_sulfur_dioxide

* Citric_acid has (+)positive relationship between fixed_acidicity
* Citric_acid has (-)negative relationship between volitile_acidicity, pH
* Citric_acid has almost no relationship between residual_sugar, free_sulfur_dioxide, total_sulfur_dioxide

It seems very hard to analyze dataset like that, therefore, it is better to go deep down analyzing with some other visualizations.

In [ ]:
fig, axes = plt.subplots(11,11, figsize=(50,50))
for i in range(11):
    for j in range(11):
        axes[i, j].scatter(data.iloc[:,i], data.iloc[:,j], c = data.quality)
        axes[i,j].set_xlabel(data.columns[i])
        axes[i,j].set_ylabel(data.columns[j])
        axes[i,j].legend(data.quality)
plt.show()

In [ ]:
g = sns.pairplot(data, hue="quality")

In [ ]:
#How many wine quality number is realted with how many unique wines
#print(data['quality'].value_counts())
sns.barplot(data['quality'].unique(),data['quality'].value_counts())
plt.xlabel("Quality Rankings")
plt.ylabel("Number of Red Wine")
plt.title("Distribution of Red Wine Quality Ratings")
plt.show()

In [ ]:
print(data['quality'].value_counts())

In [ ]:
#Check the outliers for each feature with respect to output value
fig, ax1 = plt.subplots(4,3, figsize=(22,16))
k = 0
for i in range(4):
    for j in range(3):
        if k != 11:
            sns.boxplot('quality',data.iloc[:,k], data=data, ax = ax1[i][j])
            k += 1
plt.show()

In [ ]:
#Check the outliers for each feature with respect to output value
fig, ax1 = plt.subplots(4,3, figsize=(22,16))
k = 0
for i in range(4):
    for j in range(3):
        if k != 11:
            sns.barplot('quality',data.iloc[:,k], data=data, ax = ax1[i][j])
            k += 1
plt.show()

**Classificaiton and Cross Validation**

    1. Logistic Regression
    2. SVM
    3. Decision Tree
    4. Random Forest

In [ ]:
#Fucntion Part
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import graphviz  
from sklearn.externals.six import StringIO
from IPython.display import Image 

#Normalization ==> x_norm = (x - mean)/std 
#it gives for each value the same value intervals means between 0-1
def normalization(X):
    mean = np.mean(X)
    std = np.std(X)
    X_t = (X - mean)/std
    return X_t

#Train and Test splitting of data     
def train_test(X_t, y):
    x_train, x_test, y_train, y_test = train_test_split(X_t, y, test_size = 0.3, random_state = 42)
    print("Train:",len(x_train), " - Test:", len(x_test))
    return x_train, x_test, y_train, y_test

def grid_search(name_clf, clf, x_train, x_test, y_train, y_test):
    if name_clf == 'Logistic_Regression':
        # Logistic Regression 
        log_reg_params = {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
        grid_log_reg = GridSearchCV(LogisticRegression(), log_reg_params)
        grid_log_reg.fit(x_train, y_train)
        # We automatically get the logistic regression with the best parameters.
        log_reg = grid_log_reg.best_estimator_
        print("Best Parameters for Logistic Regression: ", grid_log_reg.best_estimator_)
        print("Best Score for Logistic Regression: ", grid_log_reg.best_score_)
        print("------------------------------------------")
        return log_reg
    
    elif name_clf == 'SVM':
        # Support Vector Classifier
        svc_params = {'C':[0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0],
                      'kernel': ['rbf', 'poly', 'sigmoid', 'linear']}
        grid_svc = GridSearchCV(SVC(), svc_params)
        grid_svc.fit(x_train, y_train)
        # SVC best estimator
        svc = grid_svc.best_estimator_
        print("Best Parameters for SVM: ", grid_svc.best_estimator_)
        print("Best Score for SVM: ", grid_svc.best_score_)
        print("------------------------------------------")
        return svc
    
    elif name_clf == 'Decision_Tree':
        # DecisionTree Classifier
        tree_params = {"criterion": ["gini", "entropy"], "max_depth": list(range(2,30,1)), 
                  "min_samples_leaf": list(range(5,20,1))}
        grid_tree = GridSearchCV(DecisionTreeClassifier(), tree_params)
        grid_tree.fit(x_train, y_train)
        # tree best estimator
        tree_clf = grid_tree.best_estimator_
        print("Best Parameters for Decision Tree: ", grid_tree.best_estimator_)
        print("Best Score for Decision Tree: ", grid_tree.best_score_)
        print("------------------------------------------")
        
        #FEATURE IMPORTANCE FOR DECISION TREE
        importnce = tree_clf.feature_importances_
        plt.figure(figsize=(10,10))
        plt.title("Feature Importances of Decision Tree")
        plt.barh(X_t.columns, importnce, align="center")
        
        return tree_clf
    
    elif name_clf == 'Random_Forest':
        forest_params = {"bootstrap":[True, False], "max_depth": list(range(2,10,1)), 
                  "min_samples_leaf": list(range(5,20,1))}
        grid_forest = GridSearchCV(RandomForestClassifier(), forest_params)
        grid_forest.fit(x_train, y_train)
        # forest best estimator
        forest_clf = grid_forest.best_estimator_
        print("Best Parameters for Random Forest: ", grid_forest.best_estimator_)
        print("Best Score for Random Forest: ", grid_forest.best_score_)
        print("------------------------------------------")
        
        #FEATURE IMPORTANCE FOR DECISION TREE
        importnce = forest_clf.feature_importances_
        plt.figure(figsize=(10,10))
        plt.title("Feature Importances of Random Forest")
        plt.barh(X_t.columns, importnce, align="center")
        
        return forest_clf
    
def plot_learning_curve(estimator,title, X, y, ylim=None, cv=None, n_jobs=None,
                        train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, 
                                                            n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

#Create applying classification funciton
def apply_classification(name_clf, clf, x_train, x_test, y_train, y_test):
    #Find the best parameters and get the classification with the best parameters as return valu of grid search
    grid_clf = grid_search(name_clf, clf, x_train, x_test, y_train, y_test)
    
    #Plotting the learning curve
    # score curves, each time with 30% data randomly selected as a validation set.
    cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=42)
    plot_learning_curve(grid_clf, name_clf, x_train, y_train, 
                    ylim=(0.1, 1.01), cv=cv, n_jobs=4)
    
    #Apply cross validation to estimate the skills of models with 10 split with using best parameters
    scores = cross_val_score(grid_clf, x_train, y_train, cv=10)
    print("Mean Accuracy of Cross Validation: %", round(scores.mean()*100,2))
    print("Std of Accuracy of Cross Validation: %", round(scores.std()*100))
    print("------------------------------------------")
    
    #Predict the test data as selected classifier
    clf_prediction = grid_clf.predict(x_test)
    clf1_accuracy = sum(y_test == clf_prediction)/len(y_test)
    print("Accuracy of",name_clf,":",clf1_accuracy*100)
    
    #print confusion matrix and accuracy score before best parameters
    clf1_conf_matrix = confusion_matrix(y_test, clf_prediction)
    print("Confusion matrix of",name_clf,":\n", clf1_conf_matrix)
    print("==========================================")
    return grid_clf

**Ordinal Dataset Analyze**

grid_search, cross validation, feature selection for noise, try with taking important features then look what happens... the purpose to get good accuracy classification.

In [ ]:
#Now seperate the dataset as response variable and feature variabes
X = data.drop(['quality'], axis = 1)
#y = pd.DataFrame(data['value'])
y = data['quality']

In [ ]:
#Normalization
X_t = normalization(X)
print("X_t:", X_t.shape)

#Train and Test splitting of data 
x_train, x_test, y_train, y_test = train_test(X_t, y)

**1. Logistic Regression**

The logistic regression is a predictive analysis of statistical method. Logistic regression is analyzing a dataset in which there are one or more independent variables that determine an outcome. The outcome is measured with a dichotomous variable (in which there are only two possible outcomes). It is used to predict a binary outcome (1 / 0, Yes / No, True / False) given a set of independent variables. Logistic regression is a classification algorithm used to assign observations to a discrete set of classes. For this data set, firstly using ordinal logistic regression then it is better to using binary logistic regression with modified dataset.
Formulation of the logistic regression:
P= 1/(1+ e^(-(b_0+ b_1 x)) )

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
apply_classification('Logistic_Regression', lr, x_train, x_test, y_train, y_test)

**2. SVM**

SVM Support Vector Machine is a discriminative classifier by a separating hyperplane and supervised learning technique for Machine Learning. Differently from the unsupervised learning algorithms, there is a dataset belongs to different classes(labels). Data is trained with those class labels and then it is predicted with test data set then calculate accuracy how the algorithm predicts test data correctly. In other words, separating dataset into labeled training and test(categorize) dataset with labeled data, it could be better with work with as binary classification.
In more detail, SVM uses margin and hyperplane instead of line to separate data into two or more different class. In order to separate classes, it can be drawn many different lines but by choosing best line it is considered that margin should be maximum in between support vectors which are the closest points with different classes. While SVM algorithm is working, it follows two rules which are firstly classify correctly, then increase the margins in hyperplane. Additionally, SVM is useful for non-linear classification by increasing dimension then find a new line classification in increased dimension.
The C parameter tells the SVM optimization how many misclassifying point of each training example. For large values of C, the optimization will choose a smaller-margin hyperplane if that hyperplane does a better job of getting all the training points classified correctly. Conversely, a very small value of C will cause the optimizer to look for a larger-margin separating hyperplane, even if that hyperplane misclassifies more points. For very tiny values of C, there are misclassified examples, often even if training data is linearly separable.

In [ ]:
from sklearn.svm import SVC

svm = SVC()
apply_classification('SVM', svm, x_train, x_test, y_train, y_test)

**3. Decision Tree**

Tree models where the target variable can take a discrete set of values are called classification trees; in these tree structures, leaves represent class labels and branches represent conjunctions of features that lead to those class labels. The standard deviation is used to calculate the homogeneity of a numerical sample. After each standard deviation calculations, standard deviation reduction is used to classify dataset. The standard deviation reduction is based on the decrease in standard deviation after a dataset is split on an attribute. Constructing a decision tree is all about finding attribute that returns the highest standard deviation reduction.
A decision tree is drawn upside down with its root at the top involves partitioning the data into subsets that contain instances with similar values (homogenous), then on the middle there are condition/internal node based on the tree split into branches/edges. The end of the branch that doesn’t split anymore it is the decision/leaf tree, means that they are the last classification nodes(qualities). The base algorithm of the decision tree; recursive binary splitting. In this procedure, all the features are considered and different split points are tried and tested using a cost function. The split with the best cost (or lowest cost) is selected. The cost function is used to understand how model split and predict the split dataset classifications.

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree

dt = DecisionTreeClassifier()
dt_clf = apply_classification('Decision_Tree', dt, x_train, x_test, y_train, y_test)

In [ ]:
#Plot the decision tree 
dot_data = export_graphviz(dt_clf, out_file=None, filled=True, rounded=True,special_characters=True)
graph = graphviz.Source(dot_data)
graph

**4. Random Forest**

Random forest --> baging pf the decision tree
Random forests construct many individual decision trees at training and it uses the simplicity of decision trees with flexibility resulting in improvement the accuracy. Predictions from all trees are pooled to make the final prediction; the mode of the classes for classification or the mean prediction for regression. As they use a collection of results to make a final decision.
Random forest algorithm contains many variables, and many categorical variables with a large number of class labels. It gives results using data sets that show a loss or unbalanced distribution. When new trees are added into the random forest, algorithm updates itself with decreasing the loss by eliminating noises.
Bootstraping the sample data(creating some mini sample dataset with less variable), then calcuating with regression to the gini then pick the highly correlated, therefore first 2-3 split will be the same because of the central limit theorem because the variance of the sum is decrreasing

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
apply_classification('Random_Forest', rf, x_train, x_test, y_train, y_test)

**Binary dataset Analyze**

This dataset can be proper for classification and regression techniques because the data is multivariate, contains numeric values as an input and output value is ordered and not balanced means that there are much more normal wines than excellent or poor ones. As it seems from the figure in above, data is not balanced and the ranges are between 3-8 score. For this reason, it is more convenient to start with making data balanced.

In [ ]:
#Add a new feature according to mean of the quality
#Good wine represented by 1, bad wine represented by 0
data['value'] = ""
data['value'] = [1 if each > 5 else 0 for each in data['quality']]

print("Good Wine Class:",data[data['value'] == 1].shape)
print("Bad Wine Class:",data[data['value'] == 0].shape)

Now my data seems with the value dimension as balanced because the count of those two classes seems very equal with each other Good wine dataset equals to 855, and bad wine dataset equals to 744. Then I can modify quality column values with by value column values

In [ ]:
#Check the outliers for each feature with respect to output value
fig, ax1 = plt.subplots(4,3, figsize=(22,16))
k = 0
for i in range(4):
    for j in range(3):
        if k != 11:
            sns.boxplot('value',data.iloc[:,k], data=data, ax = ax1[i][j])
            k += 1
plt.show()

In [ ]:
#Categorical distribution plots:
fig, ax1 = plt.subplots(4,3, figsize=(22,16))
k = 0
for i in range(4):
    for j in range(3):
        if k != 11:
            sns.barplot(x="value",y=data.iloc[:,k],hue = 'value', data=data, ax = ax1[i][j])
            k += 1
plt.show()

In [ ]:
fig, axes = plt.subplots(11,11, figsize=(50,50))
for i in range(11):
    for j in range(11):
        axes[i, j].scatter(data.iloc[:,i], data.iloc[:,j], c = data.value)
        axes[i,j].set_xlabel(data.columns[i])
        axes[i,j].set_ylabel(data.columns[j])
        axes[i,j].legend(data.value)
plt.show()

In [ ]:
#Now seperate the dataset as response variable and feature variabes
Xb = data.drop(['quality','value'], axis = 1)
#y = pd.DataFrame(data['value'])
yb = data['value']

In [ ]:
#Normalization
Xb_t = normalization(Xb)
print("X_t:", Xb_t.shape)

#Train and Test splitting of data 
xb_train, xb_test, yb_train, yb_test = train_test(Xb_t, yb)

In [ ]:
lrb = LogisticRegression()
apply_classification('Logistic_Regression', lrb, xb_train, xb_test, yb_train, yb_test)

svmb = SVC()
apply_classification('SVM', svmb, xb_train, xb_test, yb_train, yb_test)

dtb = DecisionTreeClassifier()
dtb_clf = apply_classification('Decision_Tree', dtb, xb_train, xb_test, yb_train, yb_test)

rfb = RandomForestClassifier(n_estimators=100)
apply_classification('Random_Forest', rfb, xb_train, xb_test, yb_train, yb_test)

In [ ]:
#Plot the decision tree 
dot_data = export_graphviz(dtb_clf, out_file=None, filled=True, rounded=True,special_characters=True)
graph = graphviz.Source(dot_data)
graph

**3 Categorical Analysis**

In [ ]:
#Add a new feature according to mean of the quality
#Good wine represented by 2, average 1, and bad wine represented by 0
data['value'] = ""
data['value'] = [2 if each > 6 else 1 if ((each > 4) and (each < 7)) else 0 for each in data['quality']]

print("Good Wine Class:",data[data['value'] == 2].shape)
print("Average Wine Class:",data[data['value'] == 1].shape)
print("Bad Wine Class:",data[data['value'] == 0].shape)

In [ ]:
#Check the outliers for each feature with respect to output value
fig, ax1 = plt.subplots(4,3, figsize=(22,16))
k = 0
for i in range(4):
    for j in range(3):
        if k != 11:
            sns.boxplot('value',data.iloc[:,k], data=data, ax = ax1[i][j])
            k += 1
plt.show()

In [ ]:
#Categorical distribution plots:
fig, ax1 = plt.subplots(4,3, figsize=(22,16))
k = 0
for i in range(4):
    for j in range(3):
        if k != 11:
            sns.barplot(x="value",y=data.iloc[:,k],hue = 'value', data=data, ax = ax1[i][j])
            k += 1
plt.show()

In [ ]:
fig, axes = plt.subplots(11,11, figsize=(50,50))
for i in range(11):
    for j in range(11):
        axes[i, j].scatter(data.iloc[:,i], data.iloc[:,j], c = data.value)
        axes[i,j].set_xlabel(data.columns[i])
        axes[i,j].set_ylabel(data.columns[j])
        axes[i,j].legend(data.value)
plt.show()

In [ ]:
#Now seperate the dataset as response variable and feature variabes
X3 = data.drop(['quality','value'], axis = 1)
#y = pd.DataFrame(data['value'])
y3 = data['value']

In [ ]:
#Normalization
X3_t = normalization(X3)
print("X_t:", X3_t.shape)

#Train and Test splitting of data 
x3_train, x3_test, y3_train, y3_test = train_test(X3_t, y3)

**Apply all previous classification algorithms**

In [ ]:
lr3 = LogisticRegression()
apply_classification('Logistic_Regression', lr3, x3_train, x3_test, y3_train, y3_test)

svm3 = SVC()
apply_classification('SVM', svm3, x3_train, x3_test, y3_train, y3_test)

dt3 = DecisionTreeClassifier()
dt3_clf = apply_classification('Decision_Tree', dt3, x3_train, x3_test, y3_train, y3_test)

rf3 = RandomForestClassifier(n_estimators=100)
apply_classification('Random_Forest', rf3, x3_train, x3_test, y3_train, y3_test)

In [ ]:
#Plot the decision tree 
dot_data = export_graphviz(dt3_clf, out_file=None, filled=True, rounded=True,special_characters=True)
graph = graphviz.Source(dot_data)
graph

**Conclusion**


For this work, it was aimed that the analyzing which psychochemical are more related with wine quality and which approach is good for prediction of wine quality better. After all work, it is obvious that working with binary classification is more better the predict good or bad wines. During this research, four important machine learning techniques was used;

--> Logistic regression

--> Support Vector Machine 

--> Decision Tree

--> Random Forest

From all algorithms, it was obvious that for this dataset, SVM and then Random Forest algorithm gave the best model and accuracy means that those algorithms predict correctly test data. If someone wants to analyze similar data like that it is better to work SVM or Random Forest. Hence, those algorithms variances are found better with high margin terminology, therefore with multiclass analysis, those algorithms will give the best accuracy.
After the analysis of this dataset, some features have more effect to deciding quality of the wine, there are some insights about the criteria about wine quality, you can compare just looking the some psychochemical on the label of wines;
*Should be higher;*

** Alcohol is the most important feature to decide quality of the wine. If the alcohol percentage is high enough, it means that quality of the wine should be better

** Sulphates is another selecting criteria for good wines, with high percentage sulphates wine quality is increasing

** Citric Acid is another selecting criteria, it should be higher to decide more better wine

*Should be lower;*

** Volatile Acidity should be less in the good wine

** Sulfur dioxide is another effect to decreasing wine quality and also it causes head ache therefore if
there is less sulfur dioxide in wine, it should be selected

** Chlorides value has very less effect to quality of the wine but again it is obvious more value of it
causes bad quality of the wine

Additionally, for marketing point of view, if a customer wants to buy a wine just looking with some psychochemical values can decide what s/he needs to buy. Of course, brand and price feature was evaluated on this research, therefore, it is not a good analysis for saying “it is good wine”. However, it can give some idea for the people who do not have more knowledge about wine for selecting the good wine maybe for just dinner or gift for friends!